# 🔐 Divine Convergence Soulmate Parser – Starter Kit

## Step 0: Upload Your Firebase Key
Upload your `.json` service account key from Firebase.

In [ ]:

from google.colab import files
uploaded = files.upload()

import json
import os

for fn in uploaded.keys():
    if fn.endswith('.json'):
        firebase_key_path = fn
        break


## Step 1: Install Required Packages

In [ ]:
!pip install firebase-admin beautifulsoup4 requests

## Step 2: Initialize Firebase

In [ ]:

import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate(firebase_key_path)
firebase_admin.initialize_app(cred)
db = firestore.client()


## Step 3: Generate Astro-Seek URL

In [ ]:

def generate_astroseek_url(name, day, month, year, hour, minute, city):
    base = "https://horoscopes.astro-seek.com/birth-chart-horoscope-online"
    return f"{base}?narozeni_den={day}&narozeni_mesic={month}&narozeni_rok={year}&narozeni_hodina={hour}&narozeni_minuta={minute}&narozeni_mesto={city.replace(' ', '+')}&send=Send"


## Step 4: Scrape D1 Chart

In [ ]:

import requests
from bs4 import BeautifulSoup

def parse_d1_planets(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    planets = {}
    for row in soup.select('table.tranzit tr'):
        cells = row.find_all('td')
        if len(cells) >= 2:
            name = cells[0].text.strip()
            sign = cells[1].text.strip()
            if name in ['Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Pluto']:
                planets[name] = sign
    return planets


## Step 5: Push to Firebase

In [ ]:

def push_to_firebase(user_id, birth_data, chart_data):
    user_ref = db.collection('users').document(user_id)
    user_ref.set({
        'birthData': birth_data,
        'chartData': chart_data,
        'matchStatus': 'waiting'
    })
    print(f"Pushed {user_id} to Firebase.")


## Step 6: Example Usage

In [ ]:

name = "Test User"
birth_data = {
    'name': name,
    'day': 2,
    'month': 7,
    'year': 1985,
    'hour': 5,
    'minute': 25,
    'city': 'Baltimore'
}

url = generate_astroseek_url(name, **birth_data)
print("Chart URL:", url)

chart_data = parse_d1_planets(url)
print("Chart Data:", chart_data)

push_to_firebase("test_user_001", birth_data, chart_data)
